# Detailed description of software protocol
## A fully reproducible one-stop-shop for the analysis of iTRAQ/TMT data

### Folders
TODO

### Parameters
#### Folders and database search
*Folder for spectra files (files need to be mgf) and fasta database:* 
We allow the folder _IN_ and _data_ where the latter is automatically mapped to the directory from which IsoLabeledProtocol is called (when using _run.sh_ or _run.bat_). The folder should contain the spectra files and a database of protein sequences given in a FASTA file. Sequences headers should be formatted in a [UniProt](http://uniprot.org)-like format. See more details on the FASTA format in the [SearchGUI documentation]().
Raw files (e.g. Thermo raw files) can be converted into mgf using the msconvert tool of [ProteoWizard](http://proteowizard.sourceforge.net/tools.shtml). This requires a Windows computer. Files from different MS runs should be organized in folders as illustrated in Figure 1.






![Illustration of folder structure](misc/ExperimentalDesigns.svg)
**Figure 1**: Illustration of experimental designs and folder structure.






#### Experimental design

#### Quantitative analysis



### Output files and figures


### Code structure and scripts
#### Access

#### Parameter interface

#### R scripts
